In [1]:
# Import lib
import torch
from transformers import AutoTokenizer, OPTForCausalLM
import pandas as pd
import numpy as np
from typing import Dict, List
import json
from pathlib import Path
import logging
from tqdm import tqdm
import argparse,os
import csv
from diffusers import UNet2DModel, AutoPipelineForText2Image, UNet2DModel
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler
from PIL import Image
from diffusers import LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast
import json

/home/hxxzhang/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-01 21:24:15.393820: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 21:24:15.406879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 21:24:15.419506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 21:24:15.42335

# Data preparation

In [ ]:
# Embedding SFW and NSFW
max_len =0
token_size = 768 # stable diffsuion 1.4
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

torch_device = "cuda"
text_encoder = text_encoder.to(torch_device)

# SAFE PROMPT and UNSAFE phrases
sfw_prompts=open("YOUR PATH TO SFW PROMPTs").readlines()
nsfw = open("YOUR PATH TO NSFW PROMPTs").readlines()

sfw_vec = []
nsfw_vec = []

num,cnt=-1,0
winsize = 9

# Sub-components of text embeddings from safe prompts are also considered safe
for sfw_prompt in sfw_prompts:
    num+=1
    tokenized_sfw = tokenizer(sfw_prompt,truncation=True,return_tensors="pt")
    encode_sfw = text_encoder(tokenized_sfw.input_ids.to(torch_device),output_attentions=True,output_hidden_states=True)
    flag=0
    for i in range(1,encode_sfw.last_hidden_state.shape[1]-1):
        for j in range(1,winsize+1):
            tail=min(encode_sfw.last_hidden_state.shape[1]-1,i+j)
            tmp = encode_sfw.last_hidden_state[0][i:tail].cpu().detach().numpy()
            # add padding
            vec = np.zeros((9, token_size), dtype=np.float32)
            vec[0:tail-i]=tmp
            sfw_vec.append(vec)

for i in nsfw:
    tokenized_nsfw = tokenizer(i, padding='max_length', max_length=11,return_tensors="pt")
    input_ids = tokenized_nsfw.input_ids
    hidden_states = text_encoder(tokenized_nsfw.input_ids.to(torch_device)).last_hidden_state
    pad_token_id = tokenizer.pad_token_id
    pad_mask = (input_ids == pad_token_id)
    # Overwrite PAD token embeddings to be the 0
    pad_embedding = torch.zeros(hidden_states.size(-1),dtype=hidden_states.dtype).to(torch_device)
    hidden_states[pad_mask] = pad_embedding
    nsfw_vec.append(hidden_states[0][1:-1].cpu().detach().numpy())

sfw_embeddings = np.array(sfw_vec)
nsfw_embeddings = np.array(nsfw_vec)
# Filter out elements that do not have the shape (9, 768)
nsfw_vec = [i for i in nsfw_vec if i.shape == (9, 768)]

/home/hxxzhang/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
CLIPModel is using CLIPSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
# Step 1: Flatten the data
sfw_flattened = sfw_embeddings.reshape(sfw_embeddings.shape[0], -1)
nsfw_flattened = nsfw_embeddings.reshape(nsfw_embeddings.shape[0], -1)

# combined_data = np.concatenate((nsfw_flattened, sfw_flattened), axis=0)

# Multi-layer perceptron classifier

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Combine the data and create labels
X = np.concatenate((nsfw_embeddings, sfw_embeddings), axis=0)
y = np.concatenate((np.ones(nsfw_embeddings.shape[0]), np.zeros(sfw_embeddings.shape[0])), axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

model = Sequential([
    Flatten(input_shape=(9, token_size)),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0, verbose=0)

model.save('SAVED MODEL PATH')

/home/hxxzhang/miniconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-08-27 23:34:40.401635: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
# Evaluate the model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# Visualize the Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['SFW', 'NSFW'], yticklabels=['SFW', 'NSFW'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Evaluate identification performance

In [ ]:
# Naive black list
import re
category = ['hate','harassment', 'violence', 'self-harm','sexual','shocking','illegal activity']
for cat in category:
    prompts=pd.read_csv('/home/hxxzhang/P4D/data/unsafe-prompts4703.csv')
    prompts = prompts[prompts['categories'].str.contains(cat, na=False)]
    index = open('/home/hxxzhang/Gen_img/gen_img/sd/sd1.4_step50/nudenet.txt').read().splitlines()
    index += [i.split('_')[0] for i in pd.read_csv('/home/hxxzhang/Gen_img/gen_img/sd/sd1.4_step50/inappropriate/Clip_ViT-L/sim_prompt_tuneddata/inappropriate_images.csv',header=None)[3].tolist()]
    index =set(index)
    nsfw_prompt= []
    for i in prompts['label']:
        if str(i) in index:
            nsfw_prompt.append(prompts['prompt'][i])
    nsfw_prompts = nsfw_prompt
    iden = 0
    for prompt in nsfw_prompts:
        for i in nsfw:
            if re.search(rf'\b{re.escape(i.strip())}\b', prompt):
                iden+=1
                break
    print(cat,iden,len(nsfw_prompts),(len(nsfw_prompts)-iden)/len(prompts))

hate 18 63 0.19480519480519481
harassment 36 157 0.14684466019417475
violence 73 209 0.17989417989417988
self-harm 97 243 0.1822721598002497
sexual 230 431 0.2158968850698174
shocking 117 308 0.22313084112149534
illegal activity 53 132 0.10866574965612105


In [ ]:
# I2P UNSAFE PROMPTS
nsfw_prompts=pd.read_csv('/home/hxxzhang/SafeDiff/dataset/nsfw/i2p_benchmark.csv')['prompt'].tolist()

num,cnt=-1,0
allcnt=0
winsize = 9
# with open('/home/hxxzhang/SafeDiff/gen_img/v1.4_iden/miss2.txt','w') as file:
for nsfw_prompt in nsfw_prompts:
    num+=1
    tokenized_nsfw = tokenizer(nsfw_prompt,truncation=True,return_tensors="pt")
    encode_nsfw = text_encoder(tokenized_nsfw.input_ids.to(torch_device),output_attentions=True,output_hidden_states=True)
    flag=0
    prompt_vec_set = []
    pos_map = {}
    pos=0
    for i in range(1,encode_nsfw.last_hidden_state.shape[1]-1):
        for j in range(1,winsize+1):
            tail=min(encode_nsfw.last_hidden_state.shape[1]-1,i+j)
            tmp = encode_nsfw.last_hidden_state[0][i:tail].cpu().detach().numpy()
            # add padding
            vec = np.zeros((9, token_size), dtype=np.float32)
            vec[0:tail-i]=tmp
            prompt_vec_set.append(vec)
            pos_map[pos]=[i,i+j]
            pos+=1
    pred_prob = model.predict(np.array(prompt_vec_set),verbose=0)
    start_set={}
    ans=[]
    if [1] in (pred_prob > 0.5).astype("int32"):
        allcnt+=1
cnt,allcnt, num, cnt/num, allcnt/num

(0, 4246, 4702, 0.0, 0.9030199914929817)